# Introduction to XGBoost Spark with GPU

Taxi is an example of xgboost regressor. This notebook will show you how to load data, train the xgboost model and use this model to predict if a mushroom is "poisonous". Camparing to original XGBoost Spark code, there're only one API difference.

## Load libraries
First load some common libraries will be used by both GPU version and CPU version xgboost.

In [1]:
import ml.dmlc.xgboost4j.scala.spark.{XGBoostRegressor, XGBoostRegressionModel}
import org.apache.spark.sql.SparkSession
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.sql.types.{DoubleType, IntegerType, StructField, StructType}

Besides CPU version requires some extra libraries, such as:

```scala
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types.FloatType
```

## Set the dataset path

In [2]:
// You need to update them to your real paths!
val trainPath = "/data/taxi/csv/train/"
val evalPath  = "/data/taxi/csv/test/"
val transPath = "/data/taxi/csv/test/"

trainPath = /data/taxi/csv/train/
evalPath = /data/taxi/csv/test/
transPath = /data/taxi/csv/test/


/data/taxi/csv/test/

## Build the schema of the dataset
The Taxi data has 16 columns: 15 features and 1 label. "fare_amount" is the label column. The schema will be used to load data in the future. 

The next block also defines some key parameters used in xgboost training process.

In [3]:
val labelName = "fare_amount"
lazy val schema =
  StructType(Array(
    StructField("vendor_id", DoubleType),
    StructField("passenger_count", DoubleType),
    StructField("trip_distance", DoubleType),
    StructField("pickup_longitude", DoubleType),
    StructField("pickup_latitude", DoubleType),
    StructField("rate_code", DoubleType),
    StructField("store_and_fwd", DoubleType),
    StructField("dropoff_longitude", DoubleType),
    StructField("dropoff_latitude", DoubleType),
    StructField(labelName, DoubleType),
    StructField("hour", DoubleType),
    StructField("year", IntegerType),
    StructField("month", IntegerType),
    StructField("day", DoubleType),
    StructField("day_of_week", DoubleType),
    StructField("is_weekend", DoubleType)
  ))

val featureNames = schema.filter(_.name != labelName).map(_.name)

lazy val paramMap = Map(
  "learning_rate" -> 0.05,
  "max_depth" -> 8,
  "subsample" -> 0.8,
  "gamma" -> 1,
  "num_round" -> 500
)

labelName = fare_amount
schema = <lazy>
featureNames = List(vendor_id, passenger_count, trip_distance, pickup_longitude, pickup_latitude, rate_code, store_and_fwd, dropoff_longitude, dropoff_latitude, hour, year, month, day, day_of_week, is_weekend)
paramMap = <lazy>


<lazy>

## Create a new spark session and load data

A new spark session should be created to continue all the following spark operations.

NOTE: in this notebook, the dependency jars have been loaded when installing toree kernel. Alternatively the jars can be loaded into notebook by [%AddJar magic](https://toree.incubator.apache.org/docs/current/user/faq/). However, there's one restriction for `%AddJar`: the jar uploaded can only be available when `AddJar` is called just after a new spark session is created. Do it as below:

```scala
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().appName("taxi-GPU").getOrCreate
%AddJar file:/data/libs/cudf-XXX-cuda10.jar
%AddJar file:/data/libs/rapids-4-spark-XXX.jar
%AddJar file:/data/libs/xgboost4j_3.0-XXX.jar
%AddJar file:/data/libs/xgboost4j-spark_3.0-XXX.jar
// ...
```

##### Please note the new jar "rapids-4-spark-XXX.jar" is only needed for GPU version, you can not add it to dependence list for CPU version.

In [4]:
// Build the spark session and data reader as usual
val sparkSession = SparkSession.builder().appName("taxi-GPU").getOrCreate
val reader = sparkSession.read.option("header", true).schema(schema)

sparkSession = org.apache.spark.sql.SparkSession@70debb38
reader = org.apache.spark.sql.DataFrameReader@71cd4be8


org.apache.spark.sql.DataFrameReader@71cd4be8

In [5]:
val trainSet = reader.csv(trainPath)
val evalSet  = reader.csv(evalPath)
val transSet = reader.csv(transPath)

trainSet = [vendor_id: double, passenger_count: double ... 14 more fields]
evalSet = [vendor_id: double, passenger_count: double ... 14 more fields]
transSet = [vendor_id: double, passenger_count: double ... 14 more fields]


[vendor_id: double, passenger_count: double ... 14 more fields]

## Set xgboost parameters and build a XGBoostRegressor

For CPU version, `num_workers` is recommended being equal to the number of CPU cores, while for GPU version, it should be set to the number of GPUs in Spark cluster.

Besides the `tree_method` for CPU version is also different from that for GPU version. Now only "gpu_hist" is supported for training on GPU.

```scala
// difference in parameters
  "num_workers" -> 12,
  "tree_method" -> "hist",

In [6]:
val xgbParamFinal = paramMap ++ Map("tree_method" -> "gpu_hist", "num_workers" -> 1)

xgbParamFinal = Map(learning_rate -> 0.05, num_workers -> 1, subsample -> 0.8, max_depth -> 8, num_round -> 500, tree_method -> gpu_hist, gamma -> 1)


Map(learning_rate -> 0.05, num_workers -> 1, subsample -> 0.8, max_depth -> 8, num_round -> 500, tree_method -> gpu_hist, gamma -> 1)

Here comes the only API difference,`setFeaturesCol` in CPU version vs `setFeaturesCols` in GPU version.

In previous block, it said that CPU version needs `VectorAssembler` to assemble multiple feature columns into one column, because `setFeaturesCol` only accepts one feature column with the type of `vector`.

But `setFeaturesCols` supports multiple columns directly, so set the feautres column names directly to `XGBoostRegressor`. 

CPU version:

```scala
val xgbRegressor  = new XGBoostRegressor(paramMap)
  .setLabelCol(labelName)
  .setFeaturesCol("features")
```

In [7]:
val xgbRegressor = new XGBoostRegressor(xgbParamFinal)
  .setLabelCol(labelName)
  .setFeaturesCols(featureNames)

xgbRegressor = xgbr_311f9d4bf3d6


xgbr_311f9d4bf3d6

## Benchmark and train
The object `benchmark` is used to compute the elapsed time of some operations.

Training with evaluation sets is also supported in 2 ways, the same as CPU version's behavior:

* Call API `setEvalSets` after initializing an XGBoostRegressor

```scala
xgbRegressor.setEvalSets(Map("eval" -> evalSet))

```

* Use parameter `eval_sets` when initializing an XGBoostRegressor

```scala
val paramMapWithEval = paramMap + ("eval_sets" -> Map("eval" -> evalSet))
val xgbRegressorWithEval = new XGBoostRegressor(paramMapWithEval)
```

Here chooses the API way to set evaluation sets.

In [8]:
xgbRegressor.setEvalSets(Map("eval" -> evalSet))

xgbr_311f9d4bf3d6

In [9]:
object Benchmark {
  def time[R](phase: String)(block: => R): (R, Float) = {
    val t0 = System.currentTimeMillis
    val result = block // call-by-name
    val t1 = System.currentTimeMillis
    println("Elapsed time [" + phase + "]: " + ((t1 - t0).toFloat / 1000) + "s")
    (result, (t1 - t0).toFloat / 1000)
  }
}

defined object Benchmark


CPU version reqires an extra step before fitting data to regressor, using `VectorAssembler` to assemble all feature columns into one column. The following code snip shows how to do the vectorizing.

```scala
object Vectorize {
  def apply(df: DataFrame, featureNames: Seq[String], labelName: String): DataFrame = {
    val toFloat = df.schema.map(f => col(f.name).cast(FloatType))
    new VectorAssembler()
      .setInputCols(featureNames.toArray)
      .setOutputCol("features")
      .transform(df.select(toFloat:_*))
      .select(col("features"), col(labelName))
  }
}

trainSet = Vectorize(trainSet, featureCols, labelName)
evalSet = Vectorize(evalSet, featureCols, labelName)
transSet = Vectorize(transSet, featureCols, labelName)

```

Fortunately `VectorAssembler` is not needed for GPU version. Just fit the loaded data directly to XGBoostRegressor.

In [10]:
// start training
val (model, _) = Benchmark.time("train") {
  xgbRegressor.fit(trainSet)
}

Tracker started, with env={DMLC_NUM_SERVER=0, DMLC_TRACKER_URI=10.19.183.78, DMLC_TRACKER_PORT=9091, DMLC_NUM_WORKER=1}


model = xgbr_311f9d4bf3d6


Elapsed time [train]: 24.673s


xgbr_311f9d4bf3d6

## Transformation and evaluation
Here uses `transSet` to evaluate our model and use some key columns to show our predictions. Finally we use `RegressionEvaluator` to calculate an overall `rmse` of our predictions.

In [11]:
// start transform
val (prediction, _) = Benchmark.time("transform") {
  val ret = model.transform(transSet).cache()
  ret.foreachPartition((_: Iterator[_]) => ())
  ret
}
prediction.select("vendor_id", "passenger_count", "trip_distance", labelName, "prediction").show(10)
val evaluator = new RegressionEvaluator().setLabelCol(labelName)
val (rmse, _) = Benchmark.time("evaluation") {
  evaluator.evaluate(prediction)
}
println(s"RMSE == $rmse")

Elapsed time [transform]: 2.297s
+-------------+---------------+-------------+-----------+------------------+
|    vendor_id|passenger_count|trip_distance|fare_amount|        prediction|
+-------------+---------------+-------------+-----------+------------------+
|1.559730423E9|            1.0|          1.1|        6.2| 5.738811492919922|
|1.559730423E9|            4.0|          2.7|        9.4| 9.137247085571289|
|1.559730423E9|            1.0|          1.5|        6.1|6.5718793869018555|
|1.559730423E9|            1.0|          4.1|       12.6|14.173762321472168|
|1.559730423E9|            1.0|          4.6|       13.4|13.105307579040527|
|1.559730423E9|            1.0|          3.6|       12.9| 13.62382698059082|
|1.559730423E9|            1.0|          7.1|       18.3|21.106311798095703|
|1.559730423E9|            1.0|          0.8|        4.5| 4.500834941864014|
|1.559730423E9|            1.0|          2.0|        7.8| 7.606266975402832|
|1.559730423E9|            2.0|          7.

prediction = [vendor_id: double, passenger_count: double ... 15 more fields]
evaluator = RegressionEvaluator: uid=regEval_cae2d8f69dea, metricName=rmse, throughOrigin=false
rmse = 3.805039746367867


3.805039746367867

## Save the model to disk and load model
Save the model to disk and then load it to memory. After that use the loaded model to do a new prediction.

In [12]:
model.write.overwrite.save("/data/model/taxi")

val modelFromDisk = XGBoostRegressionModel.load("/data/model/taxi")
val (results2, _) = Benchmark.time("transform2") {
  modelFromDisk.transform(transSet)
}
results2.select("vendor_id", "passenger_count", "trip_distance", labelName, "prediction").show(5)

Elapsed time [transform2]: 0.05s


modelFromDisk = xgbr_311f9d4bf3d6
results2 = [vendor_id: double, passenger_count: double ... 15 more fields]


+-------------+---------------+-------------+-----------+------------------+
|    vendor_id|passenger_count|trip_distance|fare_amount|        prediction|
+-------------+---------------+-------------+-----------+------------------+
|1.559730423E9|            1.0|          1.1|        6.2| 5.738811492919922|
|1.559730423E9|            4.0|          2.7|        9.4| 9.137247085571289|
|1.559730423E9|            1.0|          1.5|        6.1|6.5718793869018555|
|1.559730423E9|            1.0|          4.1|       12.6|14.173762321472168|
|1.559730423E9|            1.0|          4.6|       13.4|13.105307579040527|
+-------------+---------------+-------------+-----------+------------------+
only showing top 5 rows



[vendor_id: double, passenger_count: double ... 15 more fields]

In [13]:
sparkSession.close()